In [39]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from huggingface_hub import notebook_login
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TrainingArguments,Trainer
from datasets import Dataset,DatasetDict
import numpy as np 
import pandas as pd 
import kaggle, zipfile
import evaluate
import warnings

# Código para ignorar avisos de usuário
warnings.filterwarnings("ignore", category=UserWarning)

# Artigo 4 - Natural Language Processing

## 1. Introdução

A aula 4 abrange uma nova área do aprendizado de máquinas: o **Processamento de Linguagem Natural**.
Além disso, é utilizado pela primeira vez uma nova biblioteca ao invés do fastai, sendo ela a Transformers, do HuggingFace. 

## 2. Objetivo

Criar um modelo capaz de processar linguagem natural, me familiarizar com a biblioteca `Transformers` e com o manejo de arquivos .csv através da `Pandas`, e participar de uma competição ativa no Kaggle.

## 3. Inspiração

Para colocar em prática o que foi passado na aula 4, queria criar um modelo que pudesse ser usado em uma competição, e a que eu escolhi foi [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/competitions/nlp-getting-started/overview), que propôe a criação de um modelo capaz de dizer se um tweet se refere a um disastre ou não.

<img src="assets/desc.png" alt="" width="600" /><br>


## 4. Execução
### 4.1 Criando DataFrame e dataset
Para começar, é necessario criar uma chave de API no Kaggle, que será baixada na forma de um arquivo `kaggle.json` que deve ser colocado no diretório `~/.kaggle/`.

Após isso podemos passar uma string com o nome do `dataset` que será usado na competição para a função `kaggle.api.competition_download_cli()`, que irá baixá-lo, e em seguida usamos a função `zipfile.ZipFile(f'{path}.zip').extractall(path)` para extrair o `dataset`.

In [2]:
path = 'nlp-getting-started'
kaggle.api.competition_download_cli(str(path))
zipfile.ZipFile(f'{path}.zip').extractall(path)

Utilizar um `!` antes do comando o faz ser executado no terminal, com isso podemos checar o conteúdo da pasta que foi baixada:

In [3]:
!ls {path}

sample_submission.csv  test.csv  train.csv


Vamos criar a variável que vai guardar o nosso DataFrame de treino:

In [4]:
df = pd.read_csv(path + '/train.csv')

DataFrame é uma tabela de colunas nomeadas, como uma tabela de banco de dados.

Usamos o atributo `head()` para ver as 5 primeiras linhas da tabela:

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Podemos usar a seguinte função para ver um exemplo de um tweet que não é sobre um desastre:

In [6]:
df[df["target"] == 0]["text"].values[1]

'I love fruits'

`df["target"] == 0` garante que será pego um valor cuja coluna `target`, que indica se o tweet é ou não sobre um desastre, seja 0, ou seja, falso.

In [7]:
df.describe(include='object')

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


Nossas colunas são `id | keyword | location | text | target`, mas como muitas entradas não possuem dados em `keyword` e `location`, escolhi remover essas colunas do DataFrame.

Isso é feito com o método `drop()`:

In [8]:
df.drop(columns = ['keyword', 'location'], inplace = True)
df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’

Também foi alterado o tipo da coluna `target` para float pois o formato anterior gerava um erro mais para frente.

A seguir criamos nosso dataset a partir do DataFrame:

In [9]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'text', 'target'],
    num_rows: 7613
})

### 4.2 Tokenização

Um modelo de deep learning espera que sua entrada sejam números, e não palavras. Por isso devemos realizar dois passos:
1. Tokenização: Dividir cada texto em partes menores denominadas *tokens*
2. Numeralização: Converter esses tokens em números

Esse processo é particular dependendo do modelo que iremos usar, então a escolha deve ser feita antes. Um modelo muito conhecido e recomendado é o seguinte:

In [10]:
model_nm = 'microsoft/deberta-v3-small'
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


A funcão `AutoTokenizer` cria um transformador de palavras em tokens.  
A seguir um exemplo de uma frase transformada em tokens:

In [11]:
tokz.tokenize("According to all known laws of aviation, ...")

['▁According',
 '▁to',
 '▁all',
 '▁known',
 '▁laws',
 '▁of',
 '▁aviation',
 ',',
 '▁.',
 '.',
 '.']

Um símbolo de underline (_) é usado para indicar o início de uma nova palavra. Palavras incomuns são divididas em pedaços.

A função a seguir *tokeniza* nossa entrada, e o `map()` executa a função em todas os itens do DataFrame.

In [12]:
def tok_func(x): return tokz(x["text"])
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

A seguir podemos ver um item do DataFrame, seu texto original e seus tokens gerados:

In [13]:
row = tok_ds[0]
row['text'], row['input_ids']

('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 [1,
  581,
  65453,
  281,
  262,
  18037,
  265,
  291,
  953,
  117831,
  903,
  4924,
  17018,
  43632,
  381,
  305,
  2])

Podemos checar qual o token para uma determinada palavra tal qual em um dicionário, através do metodo `vocab`:

In [14]:
tokz.vocab['▁are']

281

A Transformers assume que nossos rótulos (valor que queremos prever) estão na coluna com nome `labels`, por isso precisamos renomear a nossa, que é a `target`.

In [15]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

### 4.3 Conjuntos de teste e validação

#### 4.3.1 Validação

Para evitar *overfitting*, que é quando uma função fica ajustada demais ao conjunto de dados em que foi treinada e acaba desviando do formato real da função que estava sendo aproximada, é importante separar um conjunto de dados para não incluir no treinamento, pois não queremos que o modelo o veja.

Isso é chamado de *conjunto de validação*, e a biblioteca Transformers disponibiliza o `train_test_split()` para sua criação:

In [16]:
dds = tok_ds.train_test_split(0.25, seed=42) # 25% para validação e 75% para treino
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5709
    })
    test: Dataset({
        features: ['id', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1904
    })
})

Aqui o conjunto de validação é chamado `test`.

#### 4.3.2 Teste

O *conjunto de teste* também é isento do treino, e só é usado após concluído todo o processo de treinamento.

A medida em que você tenta coisas diferentes para ver o impacto nas métricas do teste de validação, é possível que encontre algumas coisas que acidentalmente melhorem os resultados naquela situação, mas que no mundo real não são exatamente melhores. Por isso o conjunto de teste serve para mostrar se ocorreu *overfitting* com o conjunto de validação.

In [17]:
eval_df = pd.read_csv(path + '/test.csv')
eval_df.drop(columns = ['keyword', 'location'], inplace = True)
eval_df['target'] = df['target'].astype(float) # previne o erro “mse_cuda” not implemented for ‘Long’
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

Chamamos de `eval` para não confundir com o outro conjunto criado acima.

### 4.4 Treinando o modelo

Primeiro definimos o *batch size*, o número de épocas, e a taxa de aprendizado:

In [19]:
bs = 64
epochs = 4
lr = 8e-5

A seguir criamos os argumentos de treinamento, contendo as variáveis definidas anteriormente e algumas configurações:

In [20]:
args = TrainingArguments(
    'outputs', 
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True, 
    evaluation_strategy="epoch", 
    per_device_train_batch_size=bs, 
    per_device_eval_batch_size=bs*2, 
    num_train_epochs=epochs, 
    weight_decay=0.01, 
    report_to='none'
    )

Agora vamos criar uma métrica para avaliar nossos resultados. Na página da competição é estipulado que a métrica avaliada é a F1:

$$F1=\frac{2*Precision*Recall}{Precision+Recall}$$

onde:

$$Precision=\frac{TP}{TP+FP}$$

$$Recall=\frac{TP}{TP+FN}$$

e:
```
True Positive [TP] = sua previsão é 1, e a correta é 1 - você preveu positivo, portanto é verdadeiro.
False Positive [FP] = sua previsão é 1, e a correta é 0 - você preveu positivo, portanto é falso.
False Negative [FN] = sua previsão é 0, e a correta é 1 - você preveu negativo, portanto é falso.
```

A biblioteca `evaluate` disponibilizada pelo HuggingFace possui várias métricas, e pode ser usada da seguinte maneira:

In [40]:
def compute_metrics(eval_preds):
    metric = evaluate.load('f1')
    logits, labels = eval_preds
    predictions = np.clip(logits, 0, 1)
    return metric.compute(predictions=predictions, references=labels)

Agora podemos criar o modelo e o `Trainer`, que funciona da mesma forma que o `Learner` no fastai:

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(
    model, 
    args, 
    train_dataset=dds['train'], 
    eval_dataset=dds['test'],
    tokenizer=tokz, 
    compute_metrics=compute_metrics
    )

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

A biblioteca Transformers gera muitas mensagens, mas podem ser ignoradas. Partimos para o treino do modelo:

In [22]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/mines/mambaforge/envs/apm-env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5709
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 360
  Number of trainable parameters = 141895681
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using th

Epoch,Training Loss,Validation Loss,F1
1,No log,0.142985,0.000000
2,No log,0.130465,0.000000
3,No log,0.131892,0.657490
4,No log,0.146977,0.742815


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation ***

Vamos ver algumas previsões que geramos:

In [29]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: id, text, target. If id, text, target are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3263
  Batch size = 128


array([[1.07128906],
       [1.08007812],
       [1.08886719],
       ...,
       [1.10449219],
       [1.09765625],
       [1.07421875]])

Como nosso gabarito não possui valores maiores do que 1 e menores do que 0, utilizamos o `np.clip()` para ficarmos com valores entre 0 e 1.

In [30]:
preds = np.clip(preds, 0, 1)
preds

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

Para submeter nossas previsões para o Kaggle, devemos gerar um arquivo .csv semelhante ao template disponibilizado, então vamos transformar os itens de `preds` em `int`:

In [34]:
preds = [int(x) for x in preds]

Agora podemos gerar um dataset a partir de um `dict` contendo o id da célula e nossa previsão, e então gerar um arquivo .csv com essas informações:

In [35]:
submission = Dataset.from_dict({
    'id': eval_ds['id'],
    'target': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

22746

Checando o arquivo gerado:

In [36]:
sub = pd.read_csv('submission.csv')
sub

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


Perfeito, agora é só submeter o arquivo na competição.

Vamos também exportar o modelo para o HuggingFace para poder ser usado por qualquer um como modelo pré-treinado:

In [38]:
model.push_to_hub("twt_disaster")

Configuration saved in /tmp/tmp2_8d0rvw/config.json
Model weights saved in /tmp/tmp2_8d0rvw/pytorch_model.bin
Uploading the following files to vnsrz/twt_disaster: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/vnsrz/twt_disaster/commit/c1a585ec407eb73b2405011e93fb83a253b23c4d', commit_message='Upload DebertaV2ForSequenceClassification', commit_description='', oid='c1a585ec407eb73b2405011e93fb83a253b23c4d', pr_url=None, pr_revision=None, pr_num=None)

## 6. Resultados

Com o arquivo enviado para o Kaggle, alcancei a posição 215 entre 866 participações:

<img src="assets/result.png" width="700" />
<img src="assets/ldb.png" width="700" /> 

O modelo está disponível no [HuggingFace](https://huggingface.co/spaces/vnsrz/disaster-recognition):

<img src="assets/hgf.png" alt="" width="700" /><br>



## 5. Conclusão

Não gostei muito da biblioteca Transformers em comparação com o fastai; os comandos são mais prolixos, e geram muitas mensagens após serem executados.

Foi difícil adaptar algumas partes da aula para o meu caso de uso, por isso precisei ir atrás de outras fontes para partes como a métrica utilizada, mas os resultados atingidos foram satisfatórios.

O que me interessou nesse artigo foi a participação na competição. A sensação de receber um problema e conseguir solucioná-lo é gratificante.